Installing TensorFlow Lite
Do we have to install the complete TensorFlow library?

Answer: NO

The only class needed to run TensorFlow Lite models is the Interpreter class.

There is a library called tflite_runtime which only have the Interpreter class.

In [2]:
!pip3 install --extra-index-url https://google-coral.github.io/py-repo tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo
     |████████████████████████████████| 1.4 MB 2.9 MB/s eta 0:00:01


In [3]:
import tflite_runtime
from tflite_runtime.interpreter import Interpreter

print(tflite_runtime.__version__)
print(tflite_runtime.interpreter.Interpreter)

2.5.0.post1
<class 'tflite_runtime.interpreter.Interpreter'>


Build the TensorFlow Lite Model
Build your own TensorFlow model and convert it into a TensorFlow Lite model.
Use an already existing TensorFlow Lite model.

Build the TensorFlow Lite Model
Build your own TensorFlow model and convert it into a TensorFlow Lite model.
Use an already existing TensorFlow Lite model.

Build TensorFlow Lite Model from Scratch
Use the TensorFlow Lite converter to convert a custom TensorFlow model into a TensorFlow Lite model.

Source: https://www.tensorflow.org/lite/convert

Use a Prebuilt TensorFlow Lite Model
There are many prebuilt TensorFlow Lite models.

https://www.tensorflow.org/lite/guide/hosted_models

In this project, a Lite version of MobileNet is used for image classification.

Download Prebuilt MobileNet Lite Model
In this project, the Lite version of a quantized MobileNet model is used for image classification.

The version used of the model accepts an input RGB image of size 224x224.

https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zi

In [5]:
import urllib.request

urllib.request.urlretrieve("https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip", "mobilenet_v1_1.0_224_quant_and_labels.zip")

('mobilenet_v1_1.0_224_quant_and_labels.zip',
 <http.client.HTTPMessage at 0x7fa387c99460>)

In [6]:
import zipfile

with zipfile.ZipFile("mobilenet_v1_1.0_224_quant_and_labels.zip", "r") as zip_file:
  zip_file.extractall("mobilenet_v1_1.0_224_quant_and_labels")

In [7]:
!ls mobilenet_v1_1.0_224_quant_and_labels

__MACOSX                          mobilenet_v1_1.0_224_quant.tflite
labels_mobilenet_quant_v1_224.txt


In [8]:
!cat mobilenet_v1_1.0_224_quant_and_labels/labels_mobilenet_quant_v1_224.txt

background
tench
goldfish
great white shark
tiger shark
hammerhead
electric ray
stingray
cock
hen
ostrich
brambling
goldfinch
house finch
junco
indigo bunting
robin
bulbul
jay
magpie
chickadee
water ouzel
kite
bald eagle
vulture
great grey owl
European fire salamander
common newt
eft
spotted salamander
axolotl
bullfrog
tree frog
tailed frog
loggerhead
leatherback turtle
mud turtle
terrapin
box turtle
banded gecko
common iguana
American chameleon
whiptail
agama
frilled lizard
alligator lizard
Gila monster
green lizard
African chameleon
Komodo dragon
African crocodile
American alligator
triceratops
thunder snake
ringneck snake
hognose snake
green snake
king snake
garter snake
water snake
vine snake
night snake
boa constrictor
rock python
Indian cobra
green mamba
sea snake
horned viper
diamondback
sidewinder
trilobite
harvestman
scorpion
black and gold garden spider
barn spider
garden spider
black widow
tarantula
wolf spider
tick
centipede
black grouse
ptarmigan
ruffed grouse
prairie chic

Make a Prediction using a TensorFlow Lite Model
Import the required libraries.
Create an instance of the Interpreter class and load the MobileNet model.
Allocate memory for the input and output tensors.
Get details about the input tensor (e.g. index and shape).
Read an input image and resize it according to the input tensor shape.
Assign the image to the input tensor.
Invoke the interpreter over the input tensor.
Get details about the output tensor (e.g. index).
Return the output of the output tensor.
Dequantize output from the output tensor.
Find the index of the score with the highest probability.

In [21]:
from tflite_runtime.interpreter import Interpreter 
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot

Create the Interpreter and Load the Model
Create an instance of the Interpreter class and load the pre-trained MobileNet TensorFlow Lite model.

In [22]:
data_folder = "mobilenet_v1_1.0_224_quant_and_labels/"

model_path = data_folder + "mobilenet_v1_1.0_224_quant.tflite"
label_path = data_folder + "labels_mobilenet_quant_v1_224.txt"


interpreter = Interpreter(model_path)
print("Model Loaded Successfully.")

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

SystemError: <built-in method CreateWrapperFromFile of PyCapsule object at 0x7fa3833f1ae0> returned a result with an error set

In [11]:
interpreter.allocate_tensors()

NameError: name 'interpreter' is not defined

In [12]:
print("Details about the input tensors:\n   ", interpreter.get_input_details())
print("There exist {num} input tensor(s).".format(num=len(interpreter.get_input_details())))
print("   Type:", type(interpreter.get_input_details()), end="\n\n")

print("Details about the first input tensor:\n  ", interpreter.get_input_details()[0])
print("   Type:", type(interpreter.get_input_details()[0]), end="\n\n")

print("Shape of the first input tensor:\n   ", interpreter.get_input_details()[0]['shape'])
print("   Type:", type(interpreter.get_input_details()[0]['shape']), end="\n\n")

_, height, width, _ = interpreter.get_input_details()[0]['shape']
print("Image Width = {width}\nImage Height = {height}".format(width=width, height=height))

NameError: name 'interpreter' is not defined

In [13]:
# Note: if no image present, upload your own test image as "test.jpg" to the mobilenet_v1 folder
image = Image.open(data_folder + "test.jpg").convert('RGB')
print("Original image size:", image.size)

image = image.resize((width, height))
print("New size:", image.size)

FileNotFoundError: [Errno 2] No such file or directory: 'mobilenet_v1_1.0_224_quant_and_labels/test.jpg'

In [14]:
def set_input_tensor(interpreter, image):

  tensor_index = interpreter.get_input_details()[0]['index']
  print("Index of the input tensor: ", tensor_index, end="\n\n")

  # Return the input tensor based on its index.
  input_tensor = interpreter.tensor(tensor_index)()[0]

  matplotlib.pyplot.subplots(nrows=1, ncols=2)
  matplotlib.pyplot.subplot(121)
  matplotlib.pyplot.imshow(input_tensor)
  matplotlib.pyplot.title("Before")

  # Assigning the image to the input tensor.
  input_tensor[:, :] = image

  matplotlib.pyplot.subplot(122)
  matplotlib.pyplot.imshow(input_tensor)
  matplotlib.pyplot.title("After")
  matplotlib.pyplot.show()

In [15]:
set_input_tensor(interpreter, image)

NameError: name 'interpreter' is not defined

In [16]:
def classify_image(interpreter, image):

  # Call the invoke() method from inside a function to avoid this RuntimeError: reference to internal data in the interpreter in the form of a numpy array or slice.
  interpreter.invoke()

  output_details = interpreter.get_output_details()[0]
  print("Details about the input tensors:\n   ", output_details, end="\n\n")

  scores = interpreter.get_tensor(output_details['index'])[0]
  print("Predicted class label score      =", np.max(np.unique(scores)))

  # Dequantize the scores.
  scale, zero_point = output_details['quantization']
  scores_dequantized = scale * (scores - zero_point)

  dequantized_max_score = np.max(np.unique(scores_dequantized))
  print("Predicted class label probability=", dequantized_max_score, end="\n\n")

  max_score_index = np.where(scores_dequantized == np.max(np.unique(scores_dequantized)))[0][0]
  print("Predicted class label ID=", max_score_index)

  return max_score_index, dequantized_max_score

In [17]:
label_id, prob = classify_image(interpreter, image)

NameError: name 'interpreter' is not defined

In [18]:
print(label_path)

with open(label_path, 'r') as f:
  labels = [line.strip() for i, line in enumerate(f.readlines())]

mobilenet_v1_1.0_224_quant_and_labels/labels_mobilenet_quant_v1_224.txt


In [19]:
# Return the classification label of the image.
classification_label = labels[label_id]

print("Image Label:", classification_label, "\nAccuracy   :", np.round(prob*100, 2), "%.")

NameError: name 'label_id' is not defined

In [20]:
from tflite_runtime.interpreter import Interpreter 
from PIL import Image
import numpy as np
import time

def set_input_tensor(interpreter, image):

  tensor_index = interpreter.get_input_details()[0]['index']
  print("Index of the input tensor: ", tensor_index, end="\n\n")

  # Return the input tensor based on its index.
  input_tensor = interpreter.tensor(tensor_index)()[0]

  # Assigning the image to the input tensor.
  input_tensor[:, :] = image

def classify_image(interpreter, image):

  set_input_tensor(interpreter, image)

  # Call the invoke() method from inside a function to avoid this RuntimeError: reference to internal data in the interpreter in the form of a numpy array or slice.
  interpreter.invoke()

  output_details = interpreter.get_output_details()[0]
  print("\nDetails about the input tensors:\n   ", output_details, end="\n\n")

  scores = interpreter.get_tensor(output_details['index'])[0]
  print("Predicted class label score      =", np.max(np.unique(scores)))

  # Dequantize the scores.
  scale, zero_point = output_details['quantization']
  scores_dequantized = scale * (scores - zero_point)

  dequantized_max_score = np.max(np.unique(scores_dequantized))
  print("Predicted class label probability=", dequantized_max_score, end="\n\n")

  max_score_index = np.where(scores_dequantized == np.max(np.unique(scores_dequantized)))[0][0]
  print("Predicted class label ID=", max_score_index)

  return max_score_index, dequantized_max_score

data_folder = "mobilenet_v1_1.0_224_quant_and_labels/"

model_path = data_folder + "mobilenet_v1_1.0_224_quant.tflite"
label_path = data_folder + "labels_mobilenet_quant_v1_224.txt"

interpreter = Interpreter(model_path)
print("Model Loaded Successfully.", end="\n\n")

interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]['shape']
print("Input tensor size: (", width, ",", height, ")")

# Load an image to be classified.
image = Image.open(data_folder + "test.jpg").convert('RGB')
print("Original image size:", image.size)

image = image.resize((width, height))
print("New image size:", image.size, end="\n\n")

# Classify the image.
label_id, prob = classify_image(interpreter, image)

# Read class labels.
with open(label_path, 'r') as f:
  labels = [line.strip() for i, line in enumerate(f.readlines())]

# Return the classification label of the image.
classification_label = labels[label_id]
print("Image Label:", classification_label, "\nAccuracy   :", np.round(prob*100, 2), "%.")

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

SystemError: <built-in method CreateWrapperFromFile of PyCapsule object at 0x7fa3833f1ae0> returned a result with an error set